In [1]:
%config Completer.use_jedi = False

In [80]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, AveragePooling2D
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow import keras
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np

In [47]:
(train_X, train_y), (testData, testLabels) = mnist.load_data()

In [48]:
print("Train Data shape : ", train_X.shape)
print("Train Labels shape : ", train_y.shape)
print("Test Data shape : ", testData.shape)
print("Test Labels shape : ", testLabels.shape)

Train Data shape :  (60000, 28, 28)
Train Labels shape :  (60000,)
Test Data shape :  (10000, 28, 28)
Test Labels shape :  (10000,)


In [49]:
train_x = tf.expand_dims(train_X, 3)

In [50]:
train_x.shape

TensorShape([60000, 28, 28, 1])

In [67]:
trainData, valData, trainLabels, valLabels = train_test_split(train_X, train_y, test_size=0.10, random_state=42)

In [68]:
# reshape image dimensions
trainData = trainData.reshape(trainData.shape[0], 28, 28, 1)
valData = valData.reshape(valData.shape[0], 28, 28, 1)
testData = testData.reshape(testData.shape[0], 28, 28, 1)

In [69]:
print("Train Data shape : ", trainData.shape)
print("Validation Data shape : ", valData.shape)
print("Test Data shape : ", testData.shape)

Train Data shape :  (54000, 28, 28, 1)
Validation Data shape :  (6000, 28, 28, 1)
Test Data shape :  (10000, 28, 28, 1)


In [70]:
# Normalization
trainData = tf.keras.utils.normalize(trainData, axis=1)
valData = tf.keras.utils.normalize(valData, axis=1)
testData = tf.keras.utils.normalize(testData, axis=1)

In [71]:
trainData[0].shape

(28, 28, 1)

In [72]:
# Le-net 5 model building
leNet_model = Sequential()
leNet_model.add(Conv2D(filters=6, kernel_size=5, strides=1, activation='tanh', input_shape=trainData[0].shape, padding='valid'))
leNet_model.add(AveragePooling2D())
leNet_model.add(Conv2D(filters=16, kernel_size=5, strides=1, activation='tanh', padding='valid'))
leNet_model.add(AveragePooling2D())
leNet_model.add(Flatten())
leNet_model.add(Dense(120, activation='tanh'))
leNet_model.add(Dense(84, activation='tanh'))
leNet_model.add(Dense(10, activation='softmax'))

In [73]:
leNet_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 24, 24, 6)         156       
_________________________________________________________________
average_pooling2d_4 (Average (None, 12, 12, 6)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 8, 16)          2416      
_________________________________________________________________
average_pooling2d_5 (Average (None, 4, 4, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 120)               30840     
_________________________________________________________________
dense_7 (Dense)              (None, 84)               

In [74]:
leNet_model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [90]:
leNet_model.fit(trainData, trainLabels, epochs=20, validation_data=(valData, valLabels))

Epoch 1/20
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0167 - accuracy: 0.9944 - val_loss: 0.0664 - val_accuracy: 0.9833
Epoch 2/20
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0162 - accuracy: 0.9947 - val_loss: 0.0632 - val_accuracy: 0.9838
Epoch 3/20
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0148 - accuracy: 0.9952 - val_loss: 0.0609 - val_accuracy: 0.9857
Epoch 4/20
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0112 - accuracy: 0.9966 - val_loss: 0.0693 - val_accuracy: 0.9823
Epoch 5/20
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0136 - accuracy: 0.9954 - val_loss: 0.0594 - val_accuracy: 0.9852
Epoch 6/20
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0098 - accuracy: 0.9967 - val_loss: 0.0684 - val_accuracy: 0.9858
Epoch 7/20
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0102 - accuracy: 0.9968 - val_loss: 0.0689 - val_accurac

In [91]:
predictions = leNet_model.predict(testData)

In [92]:
predictions

array([[6.66690824e-14, 2.59623540e-13, 2.54668300e-11, ...,
        1.00000000e+00, 6.88293931e-13, 2.32359154e-09],
       [1.16199905e-09, 5.94305549e-10, 1.00000000e+00, ...,
        4.86174518e-16, 3.53567553e-09, 4.09893083e-12],
       [7.24681003e-13, 1.00000000e+00, 4.62538905e-12, ...,
        1.26350458e-10, 8.99065961e-11, 7.19073984e-14],
       ...,
       [5.33789406e-15, 3.53671016e-11, 9.35502110e-13, ...,
        3.05012023e-11, 2.66721378e-09, 1.13646024e-07],
       [1.05468956e-09, 1.43162477e-08, 4.06732482e-13, ...,
        9.10932857e-11, 3.84049694e-04, 4.37993357e-11],
       [1.99898651e-07, 2.00905603e-13, 2.03639588e-11, ...,
        6.65820836e-16, 6.51076111e-12, 1.86876513e-12]], dtype=float32)

In [93]:
print(np.argmax(predictions[0]))

7


In [94]:
score = leNet_model.evaluate(testData, testLabels)

313/313 [==============================] - 2s 5ms/step - loss: 0.0696 - accuracy: 0.9845


In [96]:
print('Test loss: {:.3f} %'.format(score[0] * 100))
print('Test accuracy: {:.3f} %'.format(score[1] * 100))

Test loss: 6.960 %
Test accuracy: 98.450 %
